# IA Jojo

## Introdução

## Sumário

1. [Instalação de Bibliotecas](#Instalação-de-Bibliotecas)
2. [Pré processamento de Imagens](#Pré-processamento-de-Imagens)
3. [Escolha do modelo](#Escolha-do-Modelo)
4. [Treinamento do Modelo](#Treinamento-do-Modelo)
5. [Avaliação do Modelo](#Avaliação-do-Modelo)

## Instalação de Bibliotecas

Primeiramente, se ainda não foi feito, instalaremos todas as bibliotecas necessárias para esse projeto:


In [ ]:
%pip install -r requirements.txt

## Pré-processamento de Imagens

### Inicialização

Inicializamos as constantes e importamos as bibliotecas

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

# Variável para determinar o tamanho das imagens geradas no pré-processamento
IMAGE_SIZE = 224  # Você pode ajustar este valor conforme necessário

# Definindo os caminhos
CHAR_FOLDER = Path("Characters")
IMAGE_FOLDER = CHAR_FOLDER / "Image"
AUGMENTED_TRAIN_FOLDER = CHAR_FOLDER / "Augmented_Train"

# Criar diretório para imagens aumentadas, se não existir
AUGMENTED_TRAIN_FOLDER.mkdir(parents=True, exist_ok=True)


### Aplica o Data Augmentation

Pré-processamos as imagens e criamos um dataset com as transformações de data augmentation já aplicadas e salvamos em disco

In [ ]:
# Função para salvar as imagens aumentadas
def save_augmented_images(dataset, transform, save_folder):
    for idx in range(len(dataset)):
        img, label = dataset[idx]
        class_folder = save_folder / dataset.classes[label]
        class_folder.mkdir(parents=True, exist_ok=True)
        # Salvar a imagem original
        save_path = class_folder / f"{idx}_original.png"
        img_pil = transforms.ToPILImage()(img)
        img_pil.save(save_path)
        # Salvar imagens aumentadas
        for i in range(5):  # Aplicar e salvar 5 augmentações por imagem
            transformed_img = transform(img)
            save_path = class_folder / f"{idx}_{i}.png"
            transformed_img_pil = transforms.ToPILImage()(transformed_img)
            transformed_img_pil.save(save_path)

# Transformações para pré-processamento das imagens
data_transforms = {
    'augment': transforms.Compose([
        transforms.RandomResizedCrop(IMAGE_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'final': transforms.Compose([
        transforms.Resize(IMAGE_SIZE + 32),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Carregar o dataset original
original_dataset = datasets.ImageFolder(IMAGE_FOLDER)

# Salvar as imagens aumentadas
save_augmented_images(original_dataset, data_transforms['augment'], AUGMENTED_TRAIN_FOLDER)

### Inicializa o dataset principal

Inicializa o dataset de treino e teste a partir do pré-processamento e data augmentation anterior. Separamos também o dataset em 2 partes na proporção de 80% para treino e 20% para teste

In [ ]:
# Carregar o dataset aumentado
augmented_dataset = datasets.ImageFolder(AUGMENTED_TRAIN_FOLDER, transform=data_transforms['final'])

# Separar o dataset em treino (80%) e teste (20%)
train_size = int(0.8 * len(augmented_dataset))
test_size = len(augmented_dataset) - train_size
train_dataset, test_dataset = random_split(augmented_dataset, [train_size, test_size])

# Criando os data loaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'test': DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
}

# Obtendo o número de classes
class_names = augmented_dataset.classes
num_classes = len(class_names)

print(f"Classes: {class_names}")

### Exibição de imagens

Exibimos algumas imagens para ver como está o dataset

In [ ]:
# Função para mostrar imagens
def imshow(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.axis('off')

# Obtendo um batch de imagens de treino
inputs, classes = next(iter(dataloaders['train']))

# Fazendo um grid de imagens
out = utils.make_grid(inputs)

# Aumentando o tamanho da figura
plt.figure(figsize=(12, 12))

# Mostrando as imagens
imshow(out)
plt.show()  # Adiciona plt.show() para garantir que as imagens sejam exibidas em alguns ambientes

## Escolha do Modelo

### Finetune do VGG16

Nessa etapas fazemos o finetune do VGG16 congelando todas as outras camadas e modificando apenas a última para o finetune. 

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import VGG16_Weights

# Carregando o modelo VGG16 pré-treinado
model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

# Congelando os parâmetros do modelo
for param in model.parameters():
    param.requires_grad = False

# Número de características de entrada para a última camada
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, num_classes)

# Exibindo o modelo para verificar as mudanças
print(model)

## Treinamento do Modelo

### Realiza o treinamento

Realiza o treinamento e plota o gráfico dos resultados ao final e durante o treinamento, além disso ele carrega um modelo já treinado (se existir), caso não exista ele treina a partir do início, e ao final do treinamento ele salva o modelo em `model.pth`

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Definindo o dispositivo (GPU ou CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Definindo a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)

# Função para treinar o modelo
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, model_path='model.pth'):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Cada época tem uma fase de treino e uma de validação
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Definir o modelo para treinamento
            else:
                model.eval()   # Definir o modelo para validação

            running_loss = 0.0
            running_corrects = 0

            # Iterar sobre os dados
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Somente faça cálculo de gradiente na fase de treinamento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backpropagation e otimização na fase de treinamento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Armazenar a perda e precisão para plotagem
            if phase == 'train':
                train_loss_history.append(epoch_loss)
                train_acc_history.append(epoch_acc)
            else:
                val_loss_history.append(epoch_loss)
                val_acc_history.append(epoch_acc)

            # Salvar o melhor modelo
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        # Atualizar gráficos dinamicamente
        clear_output(wait=True)
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(train_loss_history, label='Train Loss')
        plt.plot(val_loss_history, label='Val Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.plot(train_acc_history, label='Train Acc')
        plt.plot(val_acc_history, label='Val Acc')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        display(plt.gcf())

    print(f'Best val Acc: {best_acc:.4f}')

    # Carregar os melhores pesos do modelo
    model.load_state_dict(best_model_wts)

    # Salvar o modelo treinado
    torch.save(model.state_dict(), model_path)

    return model

# Carregar o modelo salvo, se existir
model_path = 'model.pth'
if Path(model_path).exists():
    model.load_state_dict(torch.load(model_path))

# Treinar o modelo
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=25, model_path=model_path)


Epoch 0/24
----------
train Loss: 1.9979 Acc: 0.2277
test Loss: 1.7332 Acc: 0.2308
Epoch 1/24
----------
train Loss: 1.8952 Acc: 0.3036
test Loss: 1.6128 Acc: 0.2769
Epoch 2/24
----------
train Loss: 1.6437 Acc: 0.3527
test Loss: 1.6194 Acc: 0.3692
Epoch 3/24
----------
train Loss: 1.5659 Acc: 0.4062
test Loss: 1.5800 Acc: 0.3385
Epoch 4/24
----------
train Loss: 1.4889 Acc: 0.4554
test Loss: 1.5721 Acc: 0.3692
Epoch 5/24
----------
train Loss: 1.4049 Acc: 0.5000
test Loss: 1.4960 Acc: 0.5077
Epoch 6/24
----------
train Loss: 1.3576 Acc: 0.5312
test Loss: 1.6240 Acc: 0.3692
Epoch 7/24
----------
train Loss: 1.4255 Acc: 0.4554
test Loss: 1.5682 Acc: 0.4462
Epoch 8/24
----------
train Loss: 1.3853 Acc: 0.4732
test Loss: 1.5022 Acc: 0.4615
Epoch 9/24
----------
train Loss: 1.3990 Acc: 0.4955
test Loss: 1.4700 Acc: 0.4308
Epoch 10/24
----------
train Loss: 1.4334 Acc: 0.4241
test Loss: 1.4869 Acc: 0.4615
Epoch 11/24
----------
train Loss: 1.3199 Acc: 0.5268
test Loss: 1.3564 Acc: 0.5077
Ep

## Avaliação do Modelo

In [20]:
def evaluate_model(model, dataloaders):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloaders['test'].dataset)
    print(f'Test Accuracy: {accuracy:.4f}')

evaluate_model(model, dataloaders)

Test Accuracy: 0.5692
